# Sesión 8 — Generalización y Aprendizaje de Representaciones

Exploraremos cómo distintos modelos **organizan el espacio de entrada internamente**.

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(0)

In [ ]:

# Dataset sintético
N = 400
x0 = torch.randn(N//2, 2) + torch.tensor([-2.0, 0.0])
x1 = torch.randn(N//2, 2) + torch.tensor([ 2.0, 0.0])

X = torch.cat([x0, x1], dim=0)
y = torch.cat([torch.zeros(N//2), torch.ones(N//2)]).long()

ds = TensorDataset(X, y)
loader = DataLoader(ds, batch_size=32, shuffle=True)


In [ ]:

# Modelos: shallow vs deep

class ShallowNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )
    def forward(self, x):
        return self.net(x)

class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )
        self.classifier = nn.Linear(16, 2)

    def forward(self, x):
        h = self.features(x)
        return self.classifier(h), h


In [ ]:

# Entrenamiento simple
def train(model, epochs=200):
    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()

    for _ in range(epochs):
        for xb, yb in loader:
            opt.zero_grad()
            out = model(xb)
            if isinstance(out, tuple):
                out = out[0]
            loss = loss_fn(out, yb)
            loss.backward()
            opt.step()


In [ ]:

shallow = ShallowNet()
deep = DeepNet()

train(shallow)
train(deep)


In [ ]:

# Visualizar representaciones profundas
with torch.no_grad():
    _, h = deep(X)

plt.scatter(h[:,0], h[:,1], c=y, cmap="coolwarm")
plt.title("Representación aprendida (capa oculta)")
plt.show()



## Interpretación

- ¿Está el espacio más organizado?
- ¿Se separan mejor las clases?
- ¿Qué papel juega la profundidad?
